In [1]:

import torch
import torch.nn as nn

/home/akil/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, output_channels, kernel, stride, padding, debug=False):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, output_channels, kernel, stride, padding)
        self.batchNorm = nn.BatchNorm2d(output_channels)
        self.ReLU = nn.ReLU()
        self.debug = debug
        self.kernel = kernel
    
    def forward(self, x):
        if self.debug:
            print(f'Shape Before Convolution: {x.shape}')
        
        x = self.conv(x)
        if self.debug:
            print(f'Shape After Convolution with {self.kernel} x {self.kernel}: {x.shape}')
        
        x = self.batchNorm(x)
        
        x = self.ReLU(x)
        
        return x

In [4]:
class InceptionBlock(nn.Module):
    def __init__(self, input_channels, output_channel_list):
        super(InceptionBlock, self).__init__()
        self.path_1 = ConvBlock(input_channels, output_channel_list[0],1, 1, 0, False)
        
        self.path_2 = nn.Sequential(
            ConvBlock(input_channels, output_channel_list[1], 1, 1, 0, False),
            ConvBlock(output_channel_list[1], output_channel_list[2], 3, 1, 1, False)
        )
        
        self.path_3 = nn.Sequential(
            ConvBlock(input_channels, output_channel_list[3], 1, 1, 0, False),
            ConvBlock(output_channel_list[3], output_channel_list[4], 5, 1, 2, False)
        )
        
        self.path_4 = nn.Sequential(
            nn.MaxPool2d(3, 1, 1),
            ConvBlock(input_channels, output_channel_list[5], 1, 1, 0, False)
        )
        
    def forward(self, x):
        a = self.path_1(x)
        b = self.path_2(x) 
        c = self.path_3(x) 
        d = self.path_4(x)
        
        # print(a.shape)
        # print(b.shape)
        # print(c.shape)
        # print(d.shape)
        
        concatenated_tensor = torch.cat((a, b, c, d), dim=1)
        #print(f'Shape After Concatenation {concatenated_tensor.shape}')
        
        return concatenated_tensor

In [5]:
InceptionBlock(3, [64, 96, 128, 16, 32, 32])(torch.randn(32, 3, 224, 224)).to(device).shape

torch.Size([32, 256, 224, 224])

In [6]:
class GoogLeNet(nn.Module):
    def __init__(self, input_channels):
        super(GoogLeNet, self).__init__()
        
        self.conv1    = ConvBlock(input_channels, 64, 7, 2, 3)
        self.maxpool1 = nn.MaxPool2d(3, 2, 1)
        self.conv2    = ConvBlock(64, 192, 3, 1, 1)
        self.maxpool2 = nn. MaxPool2d(3, 2, 1)
        
        self.inception3a = InceptionBlock(192, [64, 96, 128, 16, 32, 32])
        self.inception3b = InceptionBlock(256, [128, 128, 192, 32, 96, 64])
        self.maxpool3    = nn.MaxPool2d(3, 2, 1)
        
        self.inception4a = InceptionBlock(480, [192, 96, 208, 16, 48, 64])
        self.inception4b = InceptionBlock(512, [160, 112, 224, 24, 64, 64])
        self.inception4c = InceptionBlock(512, [128, 128, 256, 24, 64, 64])
        self.inception4d = InceptionBlock(512, [112, 144, 288, 32, 64, 64])
        self.inception4e = InceptionBlock(528, [256, 160, 320, 32, 128, 128])
        self.maxpool4    = nn.MaxPool2d(3, 2, 1)
        
        self.inception5a = InceptionBlock(832, [256, 160, 320, 32, 128, 128])
        self.inception5b = InceptionBlock(832, [384, 192, 384, 48, 128, 128])
        
        self.avgPool     = nn.AvgPool2d(7, 1)
        self.dropout     = nn.Dropout(p=0.4)
        self.linear       = nn.Linear(1024, 1000)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        
        x = self.inception5a(x)
        x = self.inception5b(x)
        
        x = self.avgPool(x)
        x = self.dropout(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear(x)
        
        return x
                

In [8]:
googleNet = GoogLeNet(3)
y = googleNet(torch.randn(32, 3, 224, 224)).to("cuda")
print(y.size())

torch.Size([32, 1000])


In [17]:
x = torch.FloatTensor(28,28, 64)
y = torch.FloatTensor(28,28, 128)

torch.cat((x, y) , dim=-1).shape


torch.Size([28, 28, 192])